In [2]:
import pandas as pd
import os
import datetime as dt
pd.set_option('display.max_columns',100)
import matplotlib.pyplot as plt
import seaborn as sns
# !pip install koreanize-matplotlib
import koreanize_matplotlib
import matplotlib as mpl
pd.set_option('display.max_colwidth', None)
import glob

### enter_lesson_page

#### 기본 전처리 ()

In [85]:
enter_lesson_page = pd.read_csv('C:/Users/syc09/Desktop/코드잇/중급프로젝트1/Topic2_real/enter_lesson_page-001.csv')

C:\Users\syc09\AppData\Local\Temp\ipykernel_25876\2524196629.py:1: DtypeWarning: Columns (9,13,15) have mixed types. Specify dtype option on import or set low_memory=False.
  enter_lesson_page = pd.read_csv('C:/Users/syc09/Desktop/코드잇/중급프로젝트1/Topic2_real/enter_lesson_page-001.csv')


In [86]:
# 시간. UTC -> KST
enter_lesson_page['client_event_time'] = pd.to_datetime(enter_lesson_page['client_event_time'], errors='coerce')
enter_lesson_page['client_event_time'] = enter_lesson_page['client_event_time'] + pd.Timedelta(hours=9)

In [87]:
# 2022, 2023년 제한
enter_lesson_page = enter_lesson_page.loc[
    enter_lesson_page['client_event_time'].dt.year.between(2022, 2023)]

In [88]:
# 컬럼명 변경
enter_lesson_page.rename(columns={'content.id': 'content_id', 'lesson.id': 'lesson_id'}, inplace=True)
enter_lesson_page.rename(columns={'client_event_time': 'enter_lesson_time'}, inplace=True)

# 컬럼 버리기
enter_lesson_page.drop(columns=['platform', 'os_version', 'device_carrier'], inplace=True)
enter_lesson_page.drop(enter_lesson_page[enter_lesson_page['os_name'] == 'Chrome Headless'].index, inplace=True)

In [89]:
# 중복 컬럼 삭제
enter_lesson_page.duplicated(subset=['enter_lesson_time', 'user_id', 'content_id', 'lesson_id'], keep=False)

0           False
1           False
2           False
3           False
4           False
            ...  
21029702    False
21029703    False
21029704    False
21029705    False
21029706    False
Length: 21028196, dtype: bool

In [90]:
enter_lesson_page.head()

,city,enter_lesson_time,country,device_family,device_type,event_type,language,os_name,user_id,content_id,is_trial,lesson_id,is_free_trial
0,Wonju,2023-04-06 06:46:04.333,South Korea,Windows,Windows,enter.lesson_page,Korean,Chrome,91053eda0bffb4db6c9317acf15bc44a,94bdcbd9f329aafa84ab464b6721187d,False,d360d68b65fd3e0eafb7982386d75fd9,NaN
1,Wonju,2023-04-06 06:55:50.787,South Korea,Windows,Windows,enter.lesson_page,Korean,Chrome,91053eda0bffb4db6c9317acf15bc44a,94bdcbd9f329aafa84ab464b6721187d,False,d360d68b65fd3e0eafb7982386d75fd9,NaN
2,Wonju,2023-04-06 06:55:55.684,South Korea,Windows,Windows,enter.lesson_page,Korean,Chrome,91053eda0bffb4db6c9317acf15bc44a,94bdcbd9f329aafa84ab464b6721187d,False,770740696ab1809d6b50d4013b64a75c,NaN
3,Dongdaemun-gu,2023-04-06 06:04:55.828,South Korea,Mac,Mac,enter.lesson_page,Korean,Chrome,b2b31f00b82edc6419b25324a7ae41a4,5fc64d786416b980cfda86afd69e4516,False,68cc362ca81b5899b3c9b4dea0aaddff,NaN
4,Dongdaemun-gu,2023-04-06 06:05:34.139,South Korea,Mac,Mac,enter.lesson_page,Korean,Chrome,b2b31f00b82edc6419b25324a7ae41a4,db43a841c994231e2795d4df8931af50,False,144682f4648f81de8e6aabd11b92105e,NaN


### complete_lesson

#### 기본 전처리

In [91]:
complete_lesson = pd.read_csv('C:/Users/syc09/Desktop/코드잇/중급프로젝트1/Topic2_real/complete_lesson.csv')

In [92]:
# 시간. UTC -> KST
complete_lesson['client_event_time'] = pd.to_datetime(complete_lesson['client_event_time'], errors='coerce')
complete_lesson['client_event_time'] = complete_lesson['client_event_time'] + pd.Timedelta(hours=9)

In [93]:
# 2022, 2023년 제한
# complete_lesson = complete_lesson[(complete_lesson['client_event_time'] >= start_date) & (complete_lesson['client_event_time'] <= end_date)]
# 시간 필터링
complete_lesson = complete_lesson.loc[
    complete_lesson['client_event_time'].dt.year.between(2022, 2023)]

In [94]:
# 컬럼명 변경
complete_lesson.rename(columns={'content.id': 'content_id', 'lesson.id': 'lesson_id'}, inplace=True)
complete_lesson.rename(columns={'client_event_time': 'complete_lesson_time'}, inplace=True)

# 컬럼 버리기
complete_lesson.drop(columns=['platform', 'os_version', 'device_carrier'], inplace=True)
complete_lesson.drop(complete_lesson[complete_lesson['os_name'] == 'Chrome Headless'].index, inplace=True)

In [95]:
# 중복 컬럼 삭제
complete_lesson.duplicated(subset=['complete_lesson_time', 'user_id', 'content_id', 'lesson_id'], keep=False)

0          False
1          False
2          False
3          False
4          False
           ...  
5405064    False
5405065    False
5405066    False
5405067    False
5405068    False
Length: 5404242, dtype: bool

In [96]:
complete_lesson.head()

,city,complete_lesson_time,country,device_family,device_type,event_type,language,os_name,user_id,content_id,lesson_id
0,Wonju,2023-04-06 06:55:51.836,South Korea,Windows,Windows,complete.lesson,Korean,Chrome,91053eda0bffb4db6c9317acf15bc44a,94bdcbd9f329aafa84ab464b6721187d,d360d68b65fd3e0eafb7982386d75fd9
1,Daejeon,2023-04-06 06:19:01.243,South Korea,Windows,Windows,complete.lesson,Korean,Edge,41362ad5ebcade2bb1b78344a53e7ccf,445fac33232adcb3d1cdab7d8a579430,7e0f6aae886368ef9fa97007a8e9f7ac
2,Daejeon,2023-04-06 06:20:27.939,South Korea,Windows,Windows,complete.lesson,Korean,Edge,41362ad5ebcade2bb1b78344a53e7ccf,445fac33232adcb3d1cdab7d8a579430,d1e93c9dc08267efab31849bc6083854
3,Daejeon,2023-04-06 06:23:01.534,South Korea,Windows,Windows,complete.lesson,Korean,Edge,41362ad5ebcade2bb1b78344a53e7ccf,445fac33232adcb3d1cdab7d8a579430,f22ea5dc431035305e705552bbc4f45b
4,Daejeon,2023-04-06 06:28:34.965,South Korea,Windows,Windows,complete.lesson,Korean,Edge,41362ad5ebcade2bb1b78344a53e7ccf,445fac33232adcb3d1cdab7d8a579430,ce0bfeb7fb1259c588f437d48ab2690c


In [97]:
print(complete_lesson['complete_lesson_time'].min())
print(complete_lesson['complete_lesson_time'].max())

2022-01-01 09:00:01.181000
2023-12-31 23:59:51.635000


### 각 데이터에서 30초 이내 로그가 찍힌 것들, 동일한 년, 월, 일, 시, 분 삭제(같은 레슨일 경우)

- enter lesson

In [98]:
# enter_lesson_time을 시간 순서대로 정렬
enter_lesson_page_filtered = enter_lesson_page.sort_values(by=['user_id', 'enter_lesson_time'])

# 동일한 user_id에 대해 연속된 행의 시간 차이 계산
enter_lesson_page_filtered['time_diff'] = enter_lesson_page_filtered.groupby('user_id')['enter_lesson_time'].diff()

# 시간 차이를 초 단위로 변환
enter_lesson_page_filtered['time_diff_seconds'] = enter_lesson_page_filtered['time_diff'].dt.total_seconds() #.round(2)

In [99]:
# time_diff_seconds가 30초 초과인 데이터만 남김김
enter_lesson_page_finalised = enter_lesson_page_filtered[(enter_lesson_page_filtered['time_diff_seconds'] > 30)]

In [100]:
# 년-월-일-시-분 단위 컬럼 추가
enter_lesson_page_finalised["time_group"] = enter_lesson_page_finalised["enter_lesson_time"].dt.strftime("%Y-%m-%d %H:%M")

# 정렬 수행
enter_lesson_page_finalised = enter_lesson_page_finalised.sort_values(by=["user_id", "content_id", "lesson_id", "enter_lesson_time"])

# 중복 제거 (첫 번째 값 유지)
enter_lesson_page_finalised_unique = enter_lesson_page_finalised.drop_duplicates(subset=["user_id", "content_id", "lesson_id", "time_group"], keep="first")

enter_lesson_page_finalised_unique = enter_lesson_page_finalised_unique.drop_duplicates(
    subset=["user_id", "time_group"], keep="first"
)


# 필요 없는 컬럼 제거
enter_lesson_page_finalised_unique = enter_lesson_page_finalised_unique.drop(columns=["time_group"])


C:\Users\syc09\AppData\Local\Temp\ipykernel_25876\4201793937.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  enter_lesson_page_finalised["time_group"] = enter_lesson_page_finalised["enter_lesson_time"].dt.strftime("%Y-%m-%d %H:%M")


In [101]:
print(len(enter_lesson_page)) # 로우 데이터
print(len(enter_lesson_page_finalised)) # timediff 30초 초과 데이터
print(len(enter_lesson_page_finalised_unique)) # 동일한 년월일시분 삭제

21028196
8118190
7852494


In [102]:
print(enter_lesson_page['user_id'].nunique()) # 로우 데이터
print(enter_lesson_page_finalised['user_id'].nunique()) # timediff 30초 초과 데이터
print(enter_lesson_page_finalised_unique['user_id'].nunique()) # 동일한 년월일시분 삭제

113916
79820
79820


In [103]:
print(enter_lesson_page['lesson_id'].nunique()) # 로우 데이터
print(enter_lesson_page_finalised['lesson_id'].nunique()) # timediff 30초 초과 데이터
print(enter_lesson_page_finalised_unique['lesson_id'].nunique()) # 동일한 년월일시분 삭제

5479
5391
5383


In [104]:
enter_lesson_page_finalised_unique.describe()

,enter_lesson_time,time_diff,time_diff_seconds
count,7852494,7852494,7.852494e+06
mean,2023-01-07 20:17:21.373091584,0 days 15:31:14.171259363,5.587417e+04
min,2022-01-01 00:56:27.589000,0 days 00:00:30.001000,3.000100e+01
25%,2022-07-24 19:21:33.735750144,0 days 00:01:25.004250,8.500425e+01
50%,2023-01-26 11:07:25.428499968,0 days 00:03:20.362000,2.003620e+02
75%,2023-06-30 00:36:46.983249920,0 days 00:09:22.805000,5.628050e+02
max,2023-12-31 23:59:45.192000,722 days 23:20:52.788000,6.246485e+07
std,NaN,9 days 22:08:57.636050669,8.573376e+05


#### complete lesson

In [105]:
# enter_lesson_time을 시간 순서대로 정렬
complete_lesson_filtered = complete_lesson.sort_values(by=['user_id', 'complete_lesson_time'])

# 동일한 user_id에 대해 연속된 행의 시간 차이 계산
complete_lesson_filtered['time_diff'] = complete_lesson_filtered.groupby('user_id')['complete_lesson_time'].diff()

# 시간 차이를 초 단위로 변환
complete_lesson_filtered['time_diff_seconds'] = complete_lesson_filtered['time_diff'].dt.total_seconds() #.round(2)

In [106]:
# time_diff_seconds가 30초 초과인 데이터만 남김김
complete_lesson_finalised = complete_lesson_filtered[(complete_lesson_filtered['time_diff_seconds'] > 30)]

In [107]:
# 년-월-일-시-분 단위 컬럼 추가
complete_lesson_finalised["time_group"] = complete_lesson_finalised["complete_lesson_time"].dt.strftime("%Y-%m-%d %H:%M")

# 정렬 수행
complete_lesson_finalised = complete_lesson_finalised.sort_values(by=["user_id", "content_id", "lesson_id", "complete_lesson_time"])

# 중복 제거 (첫 번째 값 유지)
complete_lesson_finalised_unique = complete_lesson_finalised.drop_duplicates(subset=["user_id", "content_id", "lesson_id", "time_group"], keep="first")

complete_lesson_finalised_unique = complete_lesson_finalised_unique.drop_duplicates(
    subset=["user_id", "time_group"], keep="first"
)

# 필요 없는 컬럼 제거
complete_lesson_finalised_unique = complete_lesson_finalised_unique.drop(columns=["time_group"])


C:\Users\syc09\AppData\Local\Temp\ipykernel_25876\3935886420.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  complete_lesson_finalised["time_group"] = complete_lesson_finalised["complete_lesson_time"].dt.strftime("%Y-%m-%d %H:%M")


In [108]:
complete_lesson_finalised_unique.head()

,city,complete_lesson_time,country,device_family,device_type,event_type,language,os_name,user_id,content_id,lesson_id,time_diff,time_diff_seconds
222221,Seoul,2023-03-19 01:29:04.507,South Korea,Mac,Mac,complete.lesson,Korean,Safari,0000572f2ba8079bcefd2760b632e820,61b6463287573f00de13a930805a52d6,1e2a48ce24bce72e7d36da219ee9d254,0 days 00:05:53.192000,353.192
222220,Seoul,2023-03-19 01:23:11.315,South Korea,Mac,Mac,complete.lesson,Korean,Safari,0000572f2ba8079bcefd2760b632e820,61b6463287573f00de13a930805a52d6,3ae4d4504b620370687e58e0a33cf22d,0 days 00:04:43.050000,283.050
1199632,Seoul,2023-03-19 00:53:48.730,South Korea,Mac,Mac,complete.lesson,Korean,Safari,0000572f2ba8079bcefd2760b632e820,61b6463287573f00de13a930805a52d6,4d65e0729706704bf34953f2613d6ecd,0 days 00:02:14.017000,134.017
222225,Seoul,2023-03-19 01:48:55.626,South Korea,Mac,Mac,complete.lesson,Korean,Safari,0000572f2ba8079bcefd2760b632e820,61b6463287573f00de13a930805a52d6,58a55a039ef52567cfe0821f92b3c9e6,0 days 00:06:13.406000,373.406
222224,Seoul,2023-03-19 01:42:42.220,South Korea,Mac,Mac,complete.lesson,Korean,Safari,0000572f2ba8079bcefd2760b632e820,61b6463287573f00de13a930805a52d6,adfe4e8b1dc5d9f2e4b76e4bc8ee221c,0 days 00:04:11.322000,251.322


In [109]:
print(len(complete_lesson)) # 로우 데이터
print(len(complete_lesson_finalised)) # timediff 30초 초과 데이터
print(len(complete_lesson_finalised_unique)) # 동일한 년월일시분 삭제

5404242
4502312
4427700


In [110]:
print(complete_lesson['user_id'].nunique()) # 로우 데이터
print(complete_lesson_finalised['user_id'].nunique()) # timediff 30초 초과 데이터
print(complete_lesson_finalised_unique['user_id'].nunique()) # 동일한 년월일시분 삭제

49670
44419
44419


In [111]:
print(complete_lesson['lesson_id'].nunique()) # 로우 데이터
print(complete_lesson_finalised['lesson_id'].nunique()) # timediff 30초 초과 데이터
print(complete_lesson_finalised_unique['lesson_id'].nunique()) # 동일한 년월일시분 삭제

5050
5036
5034


In [112]:
complete_lesson_finalised_unique.describe()

,complete_lesson_time,time_diff,time_diff_seconds
count,4427700,4427700,4.427700e+06
mean,2023-01-21 05:03:18.948490752,0 days 15:37:22.346357195,5.624235e+04
min,2022-01-01 09:06:56.209000,0 days 00:00:30.001000,3.000100e+01
25%,2022-07-25 15:36:46.370500096,0 days 00:02:23.719000,1.437190e+02
50%,2023-01-28 16:36:47.536499968,0 days 00:04:56.299000,2.962990e+02
75%,2023-07-31 16:20:14.356499968,0 days 00:14:59.997250,8.999973e+02
max,2023-12-31 23:59:51.635000,720 days 13:46:13.325000,6.225757e+07
std,NaN,8 days 11:20:42.775909348,7.320428e+05


In [113]:
complete_lesson_finalised.describe()

,complete_lesson_time,time_diff,time_diff_seconds
count,4502312,4502312,4.502312e+06
mean,2023-01-21 03:30:51.147414016,0 days 15:28:03.565471665,5.568357e+04
min,2022-01-01 09:06:56.209000,0 days 00:00:30.001000,3.000100e+01
25%,2022-07-25 15:50:16.603500032,0 days 00:02:20.210000,1.402100e+02
50%,2023-01-28 13:08:16.047500032,0 days 00:04:51.706500,2.917065e+02
75%,2023-07-31 15:27:12.669250048,0 days 00:14:41.486500,8.814865e+02
max,2023-12-31 23:59:51.635000,720 days 13:46:13.325000,6.225757e+07
std,NaN,8 days 10:21:22.877575034,7.284829e+05


- complete_subscription에서 user_id를 가져오기

In [115]:
# complete_subscription['complete_subscription_time'].max()

In [116]:
# complete_subscription.isna().sum()

In [140]:
# 파일 불러오기 및 전처리리
complete_subscription = pd.read_csv('C:/Users/syc09/Desktop/코드잇/중급프로젝트1/Topic2_real/complete_subscription.csv')
complete_subscription['client_event_time'] = pd.to_datetime(complete_subscription['client_event_time'], errors='coerce')
complete_subscription['client_event_time'] = complete_subscription['client_event_time'] + pd.Timedelta(hours=9)
complete_subscription.drop(complete_subscription[complete_subscription['os_name'] == 'Chrome Headless'].index, inplace=True)


# 2022, 2023년 제한
# 시간 필터링
complete_subscription = complete_subscription.loc[
    complete_subscription['client_event_time'].dt.year.between(2022, 2023)]

In [141]:
complete_subscription = complete_subscription.drop_duplicates(subset=['user_id'], keep='first')
complete_subscription['user_id'].nunique()

13875

In [142]:
complete_subscription['client_event_time'].max()

Timestamp('2023-12-31 23:59:39.461000')

In [143]:
complete_subscription.isna().sum()

city                        522
client_event_time             0
country                     376
device_carrier            13875
device_family               368
device_type                 517
event_type                    0
language                    368
os_name                     374
os_version                  383
platform                    368
user_id                       0
plan.price                    0
paid_amount                   0
coupon.discount_amount        0
pg.type                   12274
dtype: int64

In [144]:
# 중복되는 user_id 제거
complete_subscription = complete_subscription.drop_duplicates(subset=['user_id'], keep='first')
complete_subscription['user_id'].nunique()

13875

In [145]:
# complete_subscription 파일에서 user_id 컬럼만 추출하여 저장
subscription_user_id_list = complete_subscription['user_id']

#### enter_lesson, complete_lesson에 subscription_user_id_list의 user_id 필터링

- enter_lesson

In [146]:
# enter_lesson_page_finalised_unique 파일에서 subscription_user_id_list에 있는 user_id만 추출하여 저장
enter_lesson_page_finalised_unique_sub = enter_lesson_page_finalised_unique[enter_lesson_page_finalised_unique['user_id'].isin(subscription_user_id_list)]
enter_lesson_page_finalised_unique_sub.head()

,city,enter_lesson_time,country,device_family,device_type,event_type,language,os_name,user_id,content_id,is_trial,lesson_id,is_free_trial,time_diff,time_diff_seconds
16138,Seoul,2023-03-18 19:42:48.654,South Korea,Mac,Mac,enter.lesson_page,Korean,Safari,0000572f2ba8079bcefd2760b632e820,61b6463287573f00de13a930805a52d6,True,09c8a2d29087cb3d47823acb5f2705e3,NaN,0 days 00:05:42.876000,342.876
6147610,Seoul,2023-03-19 00:46:47.244,South Korea,Mac,Mac,enter.lesson_page,Korean,Safari,0000572f2ba8079bcefd2760b632e820,61b6463287573f00de13a930805a52d6,False,0cf175a1781a8e0020bf6c6bdc80e78a,NaN,0 days 00:01:11.785000,71.785
6147611,Seoul,2023-03-19 00:51:34.741,South Korea,Mac,Mac,enter.lesson_page,Korean,Safari,0000572f2ba8079bcefd2760b632e820,61b6463287573f00de13a930805a52d6,False,0cf175a1781a8e0020bf6c6bdc80e78a,NaN,0 days 00:04:47.497000,287.497
1140249,Seoul,2023-03-19 01:29:04.118,South Korea,Mac,Mac,enter.lesson_page,Korean,Safari,0000572f2ba8079bcefd2760b632e820,61b6463287573f00de13a930805a52d6,False,1e2a48ce24bce72e7d36da219ee9d254,NaN,0 days 00:05:50.893000,350.893
6147608,Seoul,2023-03-19 00:45:24.704,South Korea,Mac,Mac,enter.lesson_page,Korean,Safari,0000572f2ba8079bcefd2760b632e820,61b6463287573f00de13a930805a52d6,False,2ed4b83e65edc2b1efb193aaa3c60cf9,NaN,0 days 01:12:02.068000,4322.068


In [147]:
print(len(enter_lesson_page_finalised_unique_sub))
print(enter_lesson_page_finalised_unique_sub['user_id'].nunique())

3389303
12860


In [155]:
# complete_lesson_finalised_unique 파일에서 subscription_user_id_list에 있는 user_id만 추출하여 저장
complete_lesson_finalised_unique_sub = complete_lesson_finalised_unique[complete_lesson_finalised_unique['user_id'].isin(subscription_user_id_list)]
complete_lesson_finalised_unique_sub.head()

,city,complete_lesson_time,country,device_family,device_type,event_type,language,os_name,user_id,content_id,lesson_id,time_diff,time_diff_seconds
222221,Seoul,2023-03-19 01:29:04.507,South Korea,Mac,Mac,complete.lesson,Korean,Safari,0000572f2ba8079bcefd2760b632e820,61b6463287573f00de13a930805a52d6,1e2a48ce24bce72e7d36da219ee9d254,0 days 00:05:53.192000,353.192
222220,Seoul,2023-03-19 01:23:11.315,South Korea,Mac,Mac,complete.lesson,Korean,Safari,0000572f2ba8079bcefd2760b632e820,61b6463287573f00de13a930805a52d6,3ae4d4504b620370687e58e0a33cf22d,0 days 00:04:43.050000,283.050
1199632,Seoul,2023-03-19 00:53:48.730,South Korea,Mac,Mac,complete.lesson,Korean,Safari,0000572f2ba8079bcefd2760b632e820,61b6463287573f00de13a930805a52d6,4d65e0729706704bf34953f2613d6ecd,0 days 00:02:14.017000,134.017
222225,Seoul,2023-03-19 01:48:55.626,South Korea,Mac,Mac,complete.lesson,Korean,Safari,0000572f2ba8079bcefd2760b632e820,61b6463287573f00de13a930805a52d6,58a55a039ef52567cfe0821f92b3c9e6,0 days 00:06:13.406000,373.406
222224,Seoul,2023-03-19 01:42:42.220,South Korea,Mac,Mac,complete.lesson,Korean,Safari,0000572f2ba8079bcefd2760b632e820,61b6463287573f00de13a930805a52d6,adfe4e8b1dc5d9f2e4b76e4bc8ee221c,0 days 00:04:11.322000,251.322


In [156]:
print(len(complete_lesson_finalised_unique_sub))
print(complete_lesson_finalised_unique_sub['user_id'].nunique())

1860818
12317


In [ ]:
#  첫 결제 유저 ID 목록 (중복 제거 후 리스트로 변환)
unique_first_payment_user_ids = complete_subscription['user_id'].dropna().unique().tolist()

# enter_lesson_page_finalised_unique에서 첫 결제 유저만 필터링
first_paid_enter_lesson_page_finalised_unique = enter_lesson_page_finalised_unique[enter_lesson_page_finalised_unique['user_id'].isin(unique_first_payment_user_ids)]

# 결과 출력
print("첫 결제 유저 필터링 개수:", len(first_paid_enter_lesson_page_finalised_unique))

첫 결제 유저 필터링 개수: 3389303


In [157]:
#  첫 결제 유저 ID 목록 (중복 제거 후 리스트로 변환)
unique_first_payment_user_ids = complete_subscription['user_id'].dropna().unique().tolist()

# complete_lesson_finalised_unique에서 첫 결제 유저만 필터링
first_paid_complete_lesson_finalised_unique = complete_lesson_finalised_unique[complete_lesson_finalised_unique['user_id'].isin(unique_first_payment_user_ids)]

# 결과 출력
print("첫 결제 유저 필터링 개수:", len(first_paid_complete_lesson_finalised_unique))

첫 결제 유저 필터링 개수: 1860818


#### 원찬님 방법

In [126]:
# # 임시 리스트 생성
# temp_list = []

# for data in [enter_lesson_page_df, complete_lesson_df]:
#     # 필요한 컬럼만 추출하여 필터링
#     data = data[['user_id', 'client_event_time', 'event_type', 'content.id', 'lesson.id']]

#     # 임시 리스트에 저장
#     temp_list.append(data)

# # 임시 리스트를 사용하여 데이터 concat
# lesson_concat_df = pd.concat(temp_list, axis=0)

# # reset_index()로 인덱스 재정렬
# lesson_concat_df.reset_index(drop=True)

# # user_id, client_event_time을 기준으로 오름차순 정렬
# lesson_concat_df.sort_values(by=['user_id', 'client_event_time'], ascending=[True, True])

# # 데이터 확인
# lesson_concat_df